<a href="https://colab.research.google.com/github/akhilcubettech/chatbot/blob/main/odin.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q -U torch=='2.5.0'
!pip install -q -U accelerate=='1.0.1' peft=='0.13.2' bitsandbytes=='0.44.1' transformers=='4.46.0' trl=='0.11.4' pandas=='2.2.2'


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.1/44.1 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 330.9/330.9 kB 14.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.7/320.7 kB 22.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.4/122.4 MB 17.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 123.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.6/316.6 kB 26.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 96.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 109.8/109.8 kB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 472.7/472.7 kB 38.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 17.7 MB/s eta 0:00:00


In [ ]:
!pip install wandb --quiet

In [ ]:
import os
import warnings
import pandas as pd

import wandb
import torch
import transformers
import bitsandbytes
from datasets import Dataset
from huggingface_hub import login
from peft import LoraConfig, PeftConfig, prepare_model_for_kbit_training, get_peft_model
from trl import SFTTrainer
from transformers import (AutoModelForCausalLM,
                         AutoTokenizer,
                         BitsAndBytesConfig,
                         TrainingArguments,
                         pipeline,
                         logging,
                         TrainerCallback)

from sklearn.model_selection import train_test_split
from sklearn.metrics import (accuracy_score,
                            confusion_matrix,
                            classification_report)

In [ ]:
os.environ['CUDA_VISIBLE_DEVICES'] = "0"
os.environ['TOKENIZERS_PARALLELISM'] = "false"

In [ ]:
warnings.filterwarnings("ignore")

In [ ]:
df = pd.read_csv("/content/odin.csv")
df.head()

,Context,Response
0,What industry does Jendamark Automation serve?,Jendamark Automation serves the automotive ind...
1,What kind of solutions does Jendamark Automati...,Jendamark Automation provides tech solutions t...
2,In which countries does Jendamark Automation h...,Jendamark Automation has an international pres...
3,Where does Jendamark Automation have a sales o...,Jendamark Automation has a sales office in the...
4,What does Jendamark's global reach allow them ...,Jendamark's global reach allows them to serve ...


In [ ]:
df.shape

(704, 2)

In [ ]:
df.Context[13], df.Response[13]

('Where does Jendamark Automation have an international presence?',
 'Jendamark Automation has an international presence in South Africa, India, and Germany.')

In [ ]:
def generate_prompt(datapoint):
    return f"""
            The conversation between a Human and an AI chatbot named Odinbot.
            [INST] You are a chat expert. Your task is to generate an appropriate response based on the conversation given in square brackets.
            [{datapoint['Context']}][/INST]

            {datapoint['Response']}""".strip()

def generate_test_prompt(datapoint):
    return f"""
            The conversation between a Human and an AI Chatbot named Odinbot.
            [INST] You are a chat expert. Your task is to generate an appropriate response based on the conversation given in square brackets.
            [{datapoint['Context']}][/INST]""".strip()

In [ ]:
X_train, X_eval = train_test_split(df, test_size=0.2, random_state=42)
X_train.shape, X_eval.shape

((563, 2), (141, 2))

In [ ]:
X_train = pd.DataFrame(X_train.apply(generate_prompt, axis=1), columns=['text'])
X_eval = pd.DataFrame(X_eval.apply(generate_test_prompt, axis=1),columns=['text'])
X_train.head()

,text
24,The conversation between a Human and an AI cha...
155,The conversation between a Human and an AI cha...
210,The conversation between a Human and an AI cha...
291,The conversation between a Human and an AI cha...
537,The conversation between a Human and an AI cha...


In [ ]:
train_data = Dataset.from_pandas(X_train)
eval_data = Dataset.from_pandas(X_eval)

In [ ]:
train_data[0]

{'text': 'The conversation between a Human and an AI chatbot named Odinbot.\n            [INST] You are a chat expert. Your task is to generate an appropriate response based on the conversation given in square brackets.\n            [What is the website of Jendamark?][/INST]\n\n            The website of Jendamark is www.jendamark.co.za.',
 '__index_level_0__': 24}

In [ ]:
from huggingface_hub import interpreter_login

interpreter_login()


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    A token is already saved on your machine. Run `huggingface-cli whoami` to get more information or `huggingface-cli logout` if you want to log out.
    Setting a new token will erase the existing one.
    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): ··········
Add token as git credential? (Y/n) y
Token is valid (permissi

In [ ]:
model_name = "mistralai/Mistral-7B-Instruct-v0.3"

compute_dtype = getattr(torch, "float16")

bnb_config = BitsAndBytesConfig(
    load_in_4bit = True,
    bnb_4bit_use_double_quant = False,
    bnb_4bit_quant_type = 'nf4',
    bnb_4bit_compute_dtype = compute_dtype
)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map='auto',
    quantization_config=bnb_config,
)

model.config.use_cache = False
model.config.pretraining_tp = 1

tokenizer = AutoTokenizer.from_pretrained(model_name,
                                          trust_remote_code=True,
                                          add_eos_token=True,
                                         )
tokenizer.pad_token = tokenizer.eos_token

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
model_hub_id = "akhilanilkumar/odin-bot-mistral-7b-instructv0.3-finetuned-V3"

In [ ]:
model.gradient_checkpointing_enable()
model = prepare_model_for_kbit_training(model)

In [ ]:
from google.colab import userdata
hf_key=userdata.get('hf')

In [ ]:
class PeftSavingCallback(TrainerCallback):
    def on_save(self, args, state, control, **kwargs):
        checkpoint_path = os.path.join(args.output_dir, f"checkpoint-{state.global_step}")
        kwargs["model"].save_pretrained(checkpoint_path)

        if "pytorch_model.bin" in os.listdir(checkpoint_path):
            os.remove(os.path.join(checkpoint_path, "pytorch_model.bin"))

In [ ]:
callbacks = [PeftSavingCallback]

In [ ]:
project = "odin-bot-finetuned-v2"
base_model_name = "mistral-7b-instruct"
run_name = base_model_name + "-" + project
output_dir = "./" + run_name

In [ ]:
peft_config = LoraConfig(
    lora_alpha=16,
    lora_dropout=0.05,
    r=16,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=["q_proj", "v_proj"]
)

training_arguments = TrainingArguments(
    output_dir=output_dir,
    logging_dir = "logs",
    per_device_train_batch_size=8,
    num_train_epochs=3,
    gradient_accumulation_steps=1,
    optim="paged_adamw_32bit",
    save_strategy='epoch',
    logging_steps=25,
    learning_rate=2e-4,
    weight_decay=0.001,
    fp16=True,
    bf16=False,
    max_grad_norm=0.3,
    max_steps=-1,
    warmup_ratio = 0.05,
    group_by_length=True,
    lr_scheduler_type="cosine",
    report_to="wandb",
    evaluation_strategy="epoch",
    do_eval=True,
    run_name = run_name,
    push_to_hub = True,
    hub_model_id = model_hub_id,
    hub_token=hf_key,
    hub_strategy="checkpoint",
    disable_tqdm=False
)

trainer = SFTTrainer(
    model=model,
    train_dataset=train_data,
    eval_dataset=eval_data,
    peft_config=peft_config,
    dataset_text_field="text",
    tokenizer=tokenizer,
    args=training_arguments,
    callbacks=callbacks,
    packing=False,
    max_seq_length=512)

Map:   0%|          | 0/563 [00:00<?, ? examples/s]

Map:   0%|          | 0/141 [00:00<?, ? examples/s]

In [ ]:
model = get_peft_model(model, peft_config)

In [ ]:
if torch.cuda.is_available():
    torch.cuda.empty_cache()
torch.cuda.empty_cache()

In [ ]:
from google.colab import userdata
wb_key=userdata.get('wb')

In [ ]:
wandb.login(key=wb_key)

wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


True

In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss
1,1.065600,0.851284
2,0.841200,0.818016
3,0.772600,0.818711


Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.


TrainOutput(global_step=213, training_loss=1.0707753051614537, metrics={'train_runtime': 128.1166, 'train_samples_per_second': 13.183, 'train_steps_per_second': 1.663, 'total_flos': 6843316081655808.0, 'train_loss': 1.0707753051614537, 'epoch': 3.0})

In [ ]:
trainer.push_to_hub()
wandb.finish()
model.config.use_cache = True

training_args.bin:   0%|          | 0.00/5.62k [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/27.3M [00:00<?, ?B/s]

Upload 3 LFS files:   0%|          | 0/3 [00:00<?, ?it/s]

tokenizer.model:   0%|          | 0.00/587k [00:00<?, ?B/s]

eval/loss,█▁▁█▁▁
eval/runtime,█▂▁▁▁▁
eval/samples_per_second,▁▆████
eval/steps_per_second,▁▆████
train/epoch,▁▂▃▃▄▅▅▆▇███▁▂▃▃▄▅▅▆▇███
train/global_step,▁▂▃▃▄▅▅▆▇███▁▂▃▃▄▅▅▆▇███
train/grad_norm,█▄▃▂▄▁▄▄█▄▃▂▄▁▄▄
train/learning_rate,█▇▆▅▄▃▂▁█▇▆▅▄▃▂▁
train/loss,█▂▂▂▁▁▁▁█▂▂▂▁▁▁▁
eval/loss,0.81871
eval/runtime,3.0514
